## Python在刷题中的奇技

In [ ]:
总结一些在刷LeetCode时，经常使用的一些小妙招(API)。

(只是个小白，在不断完善自己对py的使用心得。)

### 1.Lru_cache装饰器——加速缓存计算

In [ ]:
不使用lru_cache的情况下进行菲波那切数列的计算，是相当缓慢的。

In [6]:
%%time
def fib(n):
    return(1 if n in [1, 2] else fib(n-1)+fib(n-2))
print(fib(35))

9227465
Wall time: 3.07 s


而当你使用lru_cache装饰器，效果提升简直不要太明显。

In [7]:
%%time 
from functools import lru_cache
@lru_cache(None)
def fib(n):
    return (1 if n in [1, 2] else fib(n-1)+fib(n-2))
print(fib(100))

354224848179261915075
Wall time: 995 µs


 举个例子 LeetCode 97题：[交错字符串](https://leetcode-cn.com/problems/interleaving-string/description/)

你在使用回溯算法的时候，如果不使用lru一定会超时，加上缓存之后便可以轻松AC。

In [ ]:
class Solution:
    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        n1, n2, n3 = len(s1), len(s2), len(s3)
        if n1 + n2 != n3: return False
        self.res = False
        # 不加会报超时
        @functools.lru_cache(None)
        def helper(s1, s2, s3):
            if not s3:
                self.res = True
                return
            # print(s1, s2, s3)
            if s1 and s1[0] == s3[0]:
                helper(s1[1:], s2, s3[1:])
            if s2 and s2[0] == s3[0]:
                helper(s1, s2[1:], s3[1:])
            return
        helper(s1, s2, s3)
        return self.res

### 2.Counter()

如果要问我用的最多api，那么Counter必须要拥有姓名。

Counter的作用是对可迭代对象中的元素进行计数，返回一个Counter({key:value})类型。

In [11]:
from collections import Counter
nums = [1,1,1,2,3,4,5,1]
Counter(nums)

Counter({1: 4, 2: 1, 3: 1, 4: 1, 5: 1})

举个例子：LeetCode 242题 [有效的字母异位词](https://leetcode-cn.com/problems/valid-anagram/description/)

给定两个字符串 s 和 t ，编写一个函数来判断 t 是否是 s 的字母异位词。

使用Counter一行就可以搞定了。

In [ ]:
class Solution:
    def isAnagram(self, s: str, t: str) -> bool:
        return Counter(s)==Counter(t)

### 3.二分模块 bisect

很多时候自己去写一个二分法是很浪费时间的事情，在*有序序列*的情况下，直接调库是更好地选择

In [18]:
import bisect
nums = [1,2,3,4]

bisect(_left/right)可以用来在一个有序序列中，找到要插入位置的索引.

In [19]:
# bisect.bisect(nums,3) # 默认是插入相同元素的后面
bisect.bisect_left(nums,3)

2

insort(_left/right)则是找到位置直接插入。

In [23]:
bisect.insort(nums,2)
nums

[1, 2, 2, 2, 3, 4]

举个例子: 315 [计算右侧小于当前元素的个数](https://leetcode-cn.com/problems/count-of-smaller-numbers-after-self/)

给定一个整数数组 nums，按要求返回一个新数组 counts。

数组 counts 有该性质： `counts[i]` 的值是  `nums[i]` 右侧小于 `nums[i]` 的元素的数量。

In [ ]:
class Solution:
    def countSmaller(self, nums: List[int]) -> List[int]:
        q = [] # 存储和排序已经遍历过得元素
        res = []
        # 从后向前遍历，二分法查找
        for num in nums[::-1]:
            i = bisect.bisect_left(q,num)
            res.insert(0,i) # 结果也从后向前添加
            q.insert(i,num)
        return res

### 4.defaultdict()

默认的dict不是不能用，只是很多时候没有直接使用defaultdict来的方便。

举个例子：217. [存在重复元素](https://leetcode-cn.com/problems/contains-duplicate/description/)

给定一个整数数组，判断是否存在重复元素。

如果任意一值在数组中出现至少两次，函数返回 true 。如果数组中每个元素都不相同，则返回 false 。

In [ ]:
from collections import defaultdict
class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        dic = defaultdict(int)
        for i in nums:
            dic[i] += 1
            if dic[i] == 2: return True
        return False

这里如果不使用defaultdict，就要使用`dic.get(key,0)==0`来判断是否存在，

或者另个可行的办法是使用`dic.setdefault(key,0)`,都不如`defaultdict`来的方便。

### 5.OderedDict()

用来实现一个有序的字典，其实在py现有的版本中，字典已经是有序的了,那么为什么还需要ordereddict呢？

dict的有序是指每次插入键值对都会加入到字典的尾端，但是popitem()只能在尾端弹出，

而有时我们需要它在头部弹出，实现一个队列的效果——FIFO。

In [1]:
from collections import OrderedDict
d = {1:1,2:2,3:3}
od = OrderedDict(d)


In [4]:
# 从首部弹出键值对
od.popitem(last=False)

(1, 1)

### 6.product——（多个对象）的组合

从每个可迭代对象中分别取一个，返回所有可能的迭代器。

In [8]:
from itertools import product
nums1 = [1,1]
nums2 = [2,4,1]
# product()
list(product(nums1, nums2, nums2))

[(1, 2, 2),
 (1, 2, 4),
 (1, 2, 1),
 (1, 4, 2),
 (1, 4, 4),
 (1, 4, 1),
 (1, 1, 2),
 (1, 1, 4),
 (1, 1, 1),
 (1, 2, 2),
 (1, 2, 4),
 (1, 2, 1),
 (1, 4, 2),
 (1, 4, 4),
 (1, 4, 1),
 (1, 1, 2),
 (1, 1, 4),
 (1, 1, 1)]

举个例子 373.[查找和最小的K对数字](https://leetcode-cn.com/problems/find-k-pairs-with-smallest-sums/)

给定两个以升序排列的整形数组 nums1 和 nums2, 以及一个整数 k。

定义一对值 (u,v)，其中第一个元素来自 nums1，第二个元素来自 nums2。

找到和最小的 k 对数字 (u1,v1), (u2,v2) ... (uk,vk)。

In [ ]:
# 暴力枚举，排序取前k个
def kSmallestPairs(self, nums1, nums2, k):
    return sorted(itertools.product(nums1, nums2), key=sum)[:k]

### 7.permutations——（单个对象）重新组合

对可迭代对象的顺序重新组合，返回一个迭代器。

In [9]:
from itertools import permutations
citations = [3, 2, 0]
c = permutations(citations,len(citations))
print(list(c))
# citations

[(3, 2, 0), (3, 0, 2), (2, 3, 0), (2, 0, 3), (0, 3, 2), (0, 2, 3)]
